<span style="font-family:Times New Roman; font-size:14pt;">
<h2 align="center"><b>Transportation Planning Project - Phase II</b></h2>
</span>

<span style="font-family:Times New Roman; font-size:14pt;  line-height: 0.15;">
<h4 align="center"><b>Mohammadarman Maghsoudi</b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 99104265
<h4 align="center"><b>Ali Jahanshahi</b>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;   99103774
</h4>
</span>

<span style="font-family:Times New Roman; font-size:15pt;">
<h3><b>1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Problem Overview<b></h4>
<span>
<span style="font-family: Times New Roman; font-size: 13pt;">

In the project's initial phase, we forecasted bicycle rental demand at a key Seoul hub using ARIMA, Linear Regression, and Random Forest models. Analysis showed rental patterns varied by time of day, season, and day of the week. Random Forest provided the most accurate predictions. We validated models with cross-validation and hyperparameter tuning to prevent overfitting.

In the next phase, we apply these forecasts to strategically allocate bicycles. This aims to cut logistical costs and improve user satisfaction by meeting predicted demand, turning forecasts into informed decisions for operational efficiency and customer experience.

</span>

<span style="font-family:Times New Roman; font-size:15pt;">
<h3><b>2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Our Approach<b></h4>
<span>
<span style="font-family: Times New Roman; font-size: 13pt;">

decisions that enhance the overall effectiveness of the bicycle rental system.
We are using mathematical modeling and the Pulp library to solve the bicycle allocation problem optimally. This data-driven strategy aims to improve the management of shared mobility resources.

The project is structured into three steps:

1. Demand Distribution Function: We first create a function to allocate forecasted demand across stations. This tool integrates predictions into the model, ensuring efficient and responsive bike distribution.

2. Optimization Model: We mathematically define the problem's objective, constraints, and variables. We then implement this model in Pulp using a flexible, function-based design to easily test different operational scenarios.

3. Solution Implementation: We aggregate hourly forecasts into daily totals. Using a loop, we apply our functions to determine the optimal daily bike allocation for each station. This leads to data-driven decisions that balance demand fulfillment with minimal operational costs.
<span>

<span style="font-family:Times New Roman; font-size:15pt;">
<h3><b>3&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Section One; Demand Distribution for Station Pairs<b></h4>
<span>
<span style="font-family: Times New Roman; font-size: 13pt;">

In this initial step, we translate total daily rental demand into a detailed matrix of demand between station pairs. This reveals bike flow across the network for precise management.

A function performs this breakdown using statistical methods (e.g., normal, exponential distributions) to simulate realistic patterns. Its output is a dictionary mapping station pairs to their specific demand. This format is designed for direct use in our Pulp optimization model, creating a data-driven foundation for efficient bike allocation.
<span>

<span style="font-family:Times New Roman; font-size:15pt;">
<h5><b>3.1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Code Explanation<b></h4>
<span>

<span style="font-family: Times New Roman; font-size: 13pt;">

The code below defines a function named `daily_demand_distributor`, which is designed to distribute a given total daily demand for bicycle rentals among various station pairs. The function takes three parameters: `total_demand`, `num_stations`, and `distribution_method`. Here's a breakdown of the code and its functionality:

1. **Import Statements:**
   - The code begins by importing necessary libraries: `numpy` for numerical operations and `random` for generating random numbers.

2. **Function Definition - `daily_demand_distributor`:**
   - The function aims to allocate the `total_demand` among different station pairs based on the specified `distribution_method`.

3. **Random Seed:**
   - A fixed random seed (`random_seed = 42`) is set using `np.random.seed(random_seed)`. This ensures reproducibility of the results, as the same sequence of random numbers will be generated every time the function is run.

4. **Demand Distribution:**
   - The function initializes an empty dictionary `demands` to store the demand between each station pair.
   - It then checks the `distribution_method` parameter and generates a matrix of random numbers according to the specified distribution (normal, exponential, lognormal, uniform, or poisson). Each distribution method uses different parameters like `loc`, `scale`, `mean`, and `sigma` to control the characteristics of the distribution.
   - The diagonal elements of the matrix are set to 0 using `np.fill_diagonal(matrix, 0)`, ensuring that there is no demand from a station to itself.
   - The matrix is normalized so that the sum of each row equals 1, ensuring that the distribution of demand among station pairs is proportionate.

5. **Calculating Station-Pair Demands:**
   - The function iterates over the matrix and calculates the demand between each station pair `(i, j)`. This is done by multiplying the matrix element by `total_demand` and rounding the result to get an integer value.
   - These demands are stored in the `demands` dictionary with station pairs as keys.

6. **Adjusting for Total Demand:**
   - After the initial distribution, the sum of all station-pair demands might not exactly match the `total_demand` due to rounding. The code calculates the difference and adjusts the demands iteratively, either increasing or decreasing them by 1 until the total matches the `total_demand`.

7. **Return Value:**
   - Finally, the function returns the `demands` dictionary, which contains the demand between each station pair as an integer. This output is structured to be directly usable as input for the `Pulp` optimization model in the subsequent steps of the project.

This function provides a flexible and efficient way to simulate realistic demand distribution scenarios between stations, serving as a foundational step for more complex optimization tasks.

<span>

In [1]:
import numpy as np  # Importing the numpy library for numerical operations
import random  # Importing the random library for generating random numbers

# Defining a function to distribute the total daily demand for bicycle rentals among different station pairs
def daily_demand_distributor(total_demand, num_stations, distribution_method):
    
    random_seed = 42  # Setting a fixed seed for numpy's random number generator to ensure reproducibility

    np.random.seed(random_seed)  # Applying the fixed seed to numpy's random number generator

    demands = {}  # Initializing an empty dictionary to store the demand between each station pair

    # Generating a matrix of random numbers based on the specified distribution method
    if distribution_method == 'normal':
        # Creating a matrix with normally distributed random numbers
        matrix = np.random.normal(loc=0.5, scale=0.15, size=(num_stations, num_stations))
    elif distribution_method == 'exponential':
        # Creating a matrix with exponentially distributed random numbers
        matrix = np.random.exponential(scale=1.0, size=(num_stations, num_stations))
    elif distribution_method == 'lognormal':
        # Creating a matrix with log-normally distributed random numbers
        matrix = np.random.lognormal(mean=0, sigma=1, size=(num_stations, num_stations))
    elif distribution_method == 'uniform':
        # Creating a matrix with uniformly distributed random numbers between 0 and 1
        matrix = np.random.uniform(low=0, high=1, size=(num_stations, num_stations))
    elif distribution_method == 'poisson':
        # Creating a matrix with Poisson distributed random numbers with lambda=3
        matrix = np.random.poisson(lam=3, size=(num_stations, num_stations))
    else:
        # Raising an error if an unsupported distribution method is specified
        raise ValueError("Unsupported distribution method")

    np.fill_diagonal(matrix, 0)  # Setting the diagonal elements of the matrix to 0 to eliminate self-demand
    matrix = matrix.astype('float')  # Ensure matrix is of float type to handle division
    matrix /= matrix.sum(axis=1)[:, np.newaxis]  # Normalize rows to sum to 1


    # Iterating over the matrix to calculate the demand between each station pair
    for i in range(num_stations):
        for j in range(num_stations):
            if i != j:  # Ensuring that we're not considering self-demand
                prop_demand = matrix[i, j] * total_demand  # Calculating proportional demand based on the matrix value
                demands[(i, j)] = int(round(prop_demand))  # Rounding the proportional demand to the nearest integer and storing it

    demand_values = np.array(list(demands.values()))  # Creating an array of demand values to calculate the total
    difference = total_demand - demand_values.sum()  # Calculating the difference between the total demanded and the sum of distributed demands

    # Adjusting the distributed demands to ensure the sum matches the total demand
    while difference != 0:
        for (i, j) in demands:
            if difference > 0:  # If the total is greater, increment the demand between a station pair
                demands[(i, j)] += 1
                difference -= 1  # Decrease the difference
            elif difference < 0 and demands[(i, j)] > 0:  # If the total is lesser, decrement the demand if it's not already zero
                demands[(i, j)] -= 1
                difference += 1  # Increase the difference
            if difference == 0:  # Break the loop if the difference has been neutralized
                break

    return demands  # Returning the final dictionary of demands between station pairs

<span style="font-family:Times New Roman; font-size:15pt;">
<h5><b>3.2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Further Discussion<b></h4>
<span>

<span style="font-family: Times New Roman; font-size: 13pt;">

Using statistical distributions to model demand between bicycle rental stations provides a realistic and flexible approach to understanding rental patterns. Each distribution type captures different characteristics of demand variability and real-world phenomena.

**Rationale for Using Distributions:**

1. **Reflecting Real-World Variability:**
   - Bicycle rental demand is influenced by time of day, weather, events, and user behavior. Statistical distributions provide a mathematical framework to simulate this variability and model potential real-world scenarios.

2. **Modeling Different Demand Patterns:**
   - **Normal Distribution:** Models demand clustered around a mean, such as peak hours or popular stations with limited deviations.
   - **Exponential Distribution:** Suitable for modeling time between rental requests, particularly in less busy areas with spread-out arrivals.
   - **Lognormal Distribution:** Captures demand that varies widely but clusters around a lower limit, reflecting areas with differing popularity.
   - **Uniform Distribution:** Simulates a baseline scenario where demand is evenly distributed across stations without peaks.
   - **Poisson Distribution:** Ideal for modeling the count of rental requests within a fixed time interval.

3. **Facilitating Robust Optimization:**
   - Simulating demand with different distributions tests optimization models under various scenarios, ensuring the system remains resilient and efficient despite fluctuations.

4. **Enabling Statistical Analysis and Prediction:**
   - Understanding demand distributions allows for informed predictions, trend identification, and assessment of external factors on rental behavior.

5. **Customization and Flexibility:**
   - The choice of distribution can be tailored to specific data characteristics or environmental factors, ensuring model relevance and accuracy as conditions evolve.

Given that the number of stations and station-pair demands are unknown, employing statistical distributions offers a sophisticated method to model demand complexity. This approach enhances forecast accuracy and supports more effective operational planning and decision-making for bicycle rental systems.
<span>

<span style="font-family:Times New Roman; font-size:15pt;">
<h3><b>4&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Section Two; Optimization Model for Bicycle Sharing<b></h4>
<span>
<span style="font-family: Times New Roman; font-size: 13pt;">

At this stage, we focus on the smart allocation of bicycles across stations. This directly impacts our ability to meet demand while minimizing costs. A mathematical model translates this complex problem into simpler terms to identify the best solution.

Our model consists of four key components:
- **Inputs:** Data such as the number of bikes needed at each station.
- **Decision Variables:** Choices we control, like how many bikes to move between stations.
- **Objective:** Our goal, which is to minimize costs.
- **Constraints:** Limitations, such as each station's maximum capacity.

These components guide efficient bike distribution. In the next sections, we will detail how they work together in the model.
<span>

<span style="font-family:Times New Roman; font-size:15pt;">
<h5><b>4.1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Optimization Model<b></h4>
<span>

<span style="font-family: Times New Roman; font-size: 13pt;">

This section outlines the mathematical formulation used for optimizing the distribution of bicycles across a network of rental stations, aiming to minimize overall operational costs while ensuring customer demand is met within station capacity constraints.

<span style="font-family:Times New Roman; font-size:13pt;">
<h5><b>4.1.1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Parameters<b></h4>
<span>

- $D(i, j)$: Daily demand for bikes from station $i$ to station $j$
- $N$: Number of bike stations
- $C_p$: Purchase cost per bike (constant across all stations)
- $C_s(i)$: Stock-out cost at station $i$
- $C_t(i)$: Time wasted cost at station $i$
- $C_r(i, j)$: Redistribution cost from station $i$ to $j$
- $Q(i)$: Capacity of station $i$

<br>

<span style="font-family:Times New Roman; font-size:13pt;">
<h5><b>4.1.2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Decision Variables<b></h4>
<span>

- $B(i)$: Number of bikes at station $i$
- $S_{sh}(i, j)$: Stock shortage from station $i$ to $j$
- $O(i)$: Overstock at station $i$
- $D_{bn}(i, j)$: Demand balance needed from station $i$ to $j$
- $S_{sp}(i)$: Stock surplus at station $i$
- $U(i)$: Understock at station $i$
- $D_{br}(i, j)$: Demand balance received from station $i$ to $j$
- $R_{in}(i)$: Redistribution in to station $i$
- $R_{out}(i)$: Redistribution out from station $i$
- $R_{v}(i, j)$: Redistribution volume from station $i$ to $j$

<br>


<span style="font-family:Times New Roman; font-size:13pt;">
<h5><b>4.1.3&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Objective Function<b></h4>
<span>

Minimize the total cost, which includes the purchase cost of bikes, stock-out costs, time waste costs due to overstock, and redistribution costs:

$$
\min \left( \sum_{i=1}^{N} C_p \cdot B(i) + \sum_{i=1}^{N} \sum_{\substack{j=1 \\ j \neq i}}^{N} C_s(i) \cdot S_{sh}(i, j) + \sum_{i=1}^{N} C_t(i) \cdot O(i) + \sum_{i=1}^{N} \sum_{\substack{j=1 \\ j \neq i}}^{N} C_r(i, j) \cdot R_{v}(i, j) \right)
$$

<br>

<span style="font-family:Times New Roman; font-size:13pt;">
<h5><b>4.1.4&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Constraints<b></h4>
<span>

1. **Bike Station Capacity Constraint**
   - Ensures that the number of bikes at each station does not exceed the station's capacity, a physical and operational limit.
     $$ B(i) \leq Q(i) \quad \forall i $$

2. **Demand Balance Needed Calculation**
   - Calculates the additional bikes needed at station \( i \) to satisfy the daily demand to station \( j \), after accounting for any stock shortages. This reflects the requirement to meet inter-station bike demands.
     $$ D_{bn}(i, j) = D(i, j) - S_{sh}(i, j) \quad \forall i, j; i \neq j $$

3. **Stock Surplus Calculation**
   - Determines the excess bikes at station \( i \), considering bikes present and the outgoing demand, adjusted for shortages. This surplus is critical for understanding which stations have more bikes than needed to meet outgoing demand.
     $$ S_{sp}(i) - \sum_{\substack{j=1 \\ j \neq i}}^{N} S_{sh}(i, j) = B(i) - \sum_{\substack{j=1 \\ j \neq i}}^{N} D(i, j) \quad \forall i $$

4. **Overstock and Understock Balance**
   - Balances the overstock (excess bikes) and understock (deficit of bikes) at each station, considering its capacity, current bike count, and the net demand balance. This constraint helps manage the station's bike levels to neither too high (causing overstock) nor too low (causing understock).
     $$ O(i) - U(i) = Q(i) - B(i) + \sum_{j=1}^{N} D_{bn}(i, j) - \sum_{j=1}^{N} D_{bn}(j, i) \quad \forall i $$

5. **Ensuring Demand Balance Received Equals Understock**
   - Guarantees that the number of bikes received from other stations to balance demand at station \( i \) matches its understock. This constraint is pivotal for redistributing bikes to stations in need.
     $$ \sum_{j=1}^{N} D_{br}(i, j) = U(i) \quad \forall i $$

6. **Limit on Demand Balance Received by the Overstock**
   - Prevents stations from receiving more bikes than their overstock can accommodate, avoiding the situation where bikes are sent to stations that already have too many.
     $$ \sum_{j=1}^{N} D_{br}(j, i) \leq O(i) \quad \forall i $$

7. **Redistribution In and Out Balance**
   - Ensures that the net bikes redistributed to a station (accounting for both incoming and outgoing bikes) aligns with the station's bike level changes, considering overstock and received demand balance. This maintains the flow of bikes in response to supply and demand dynamics.
     $$ R_{in}(i) - R_{out}(i) = Q(i) - O(i) + \sum_{j=1}^{N} D_{br}(i, j) - B(i) \quad \forall i $$

8. **Total Redistribution Volume In Must Equal Redistribution In**
   - Confirms that the total number of bikes redistributed into a station from all others matches the calculated redistribution in volume, ensuring consistency in inbound bike movements.
     $$ \sum_{j=1}^{N} R_{v}(i, j) = R_{in}(i) \quad \forall i $$

9. **Total Redistribution Volume Out Must Equal Redistribution Out**
   - Ensures that the total number of bikes redistributed out of a station to all others matches the calculated redistribution out volume, maintaining consistency in outbound bike movements.
     $$ \sum_{j=1}^{N} R_{v}(j, i) = R_{out}(i) \quad \forall i $$

10. **10th Constraint: Non-negative Integer Variables**

  - This constraint ensures that all decision variables in the model belong to the set of non-negative integers, reflecting the discrete and non-negative nature of the quantities involved in the bike-sharing system (e.g., number of bikes, shortages, surpluses). Incorporating this constraint ensures the model's practical applicability, as it aligns with the real-world requirement that these variables represent tangible, countable items and actions within the bike-sharing system.

$$ B(i), S_{sh}(i, j), O(i), D_{bn}(i, j), S_{sp}(i), U(i), D_{br}(i, j), R_{in}(i), R_{out}(i), R_{v}(i, j) \in \mathbb{Z}^+ \quad \forall i, j; i \neq j $$


<br>

<span style="font-family:Times New Roman; font-size:13pt;">
<h5><b>4.1.5&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Final Model<b></h4>
<span>

With the foundational components in place, we can now present the complete optimization model as outlined below. This model encapsulates our objective and the constraints within a mathematical framework, providing a clear and structured approach to optimizing the distribution of bicycles across the network of rental stations.

$$
\text{Minimize} \quad Z = \sum_{i=1}^{N} C_p \cdot B(i) + \sum_{i=1}^{N} \sum_{\substack{j=1 \\ j \neq i}}^{N} C_s(i) \cdot S_{sh}(i, j) + \sum_{i=1}^{N} C_t(i) \cdot O(i) + \sum_{i=1}^{N} \sum_{\substack{j=1 \\ j \neq i}}^{N} C_r(i, j) \cdot R_{v}(i, j) 
$$

$$
s.t.
$$

$$
B(i) \leq Q(i) \quad \forall i \quad (\text{1})
$$

$$
D_{bn}(i, j) = D(i, j) - S_{sh}(i, j) \quad \forall i, j; i \neq j \quad (\text{2})
$$

$$
S_{sp}(i) - \sum_{\substack{j=1 \\ j \neq i}}^{N} S_{sh}(i, j) = B(i) - \sum_{\substack{j=1 \\ j \neq i}}^{N} D(i, j) \quad \forall i \quad (\text{3})
$$

$$
O(i) - U(i) = Q(i) - B(i) + \sum_{j=1}^{N} D_{bn}(i, j) - \sum_{j=1}^{N} D_{bn}(j, i) \quad \forall i \quad (\text{4})
$$

$$
\sum_{j=1}^{N} D_{br}(i, j) = U(i) \quad \forall i \quad (\text{5})
$$

$$
\sum_{j=1}^{N} D_{br}(j, i) \leq O(i) \quad \forall i \quad (\text{6})
$$

$$
R_{in}(i) - R_{out}(i) = Q(i) - O(i) + \sum_{j=1}^{N} D_{br}(i, j) - B(i) \quad \forall i \quad (\text{7})
$$

$$
\sum_{j=1}^{N} R_{v}(i, j) = R_{in}(i) \quad \forall i \quad (\text{8})
$$

$$
\sum_{j=1}^{N} R_{v}(j, i) = R_{out}(i) \quad \forall i \quad (\text{9})
$$

$$
B(i), S_{sh}(i, j), O(i), D_{bn}(i, j), S_{sp}(i), U(i), D_{br}(i, j), R_{in}(i), R_{out}(i), R_{v}(i, j) \in \mathbb{Z}^+ \quad \forall i, j; i \neq j \quad (\text{10})
$$

<span>

<span style="font-family:Times New Roman; font-size:15pt;">
<h5><b>4.2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Code Implementation<b></h4>
<span>

<span style="font-family: Times New Roman; font-size: 13pt;">

Now that we have our optimization model established, it's time to implement it in Python using the `pulp` library. This step translates our theoretical framework into a practical solution, enabling us to efficiently solve the bicycle distribution problem across rental stations. Here's a streamlined overview of how the code brings our model to life:

1. **Setup with `pulp`:** The code begins by importing necessary components from `pulp`, a powerful tool for linear optimization in Python, setting the stage for defining and solving our problem.

2. **Defining the Function:** The `bike_sharing_model` function is our main workhorse, taking in parameters like daily demand, station information, costs, and capacities, which are crucial for the optimization.

3. **Costs and Capacities:** Using dictionaries, the code assigns costs for procurement, stockouts, excess inventory, and redistribution, alongside station capacity limits, reflecting the real-world variability and constraints we must consider.

4. **Problem Initialization:** An `LpProblem` instance is created to represent our optimization challenge, with the goal clearly set to minimize the total operational cost.

5. **Decision Variables Creation:** The code defines several `LpVariable` instances for bikes at stations, stockouts, and redistribution actions, ensuring they are treated as discrete quantities in the optimization process.

6. **Objective Function Setup:** The total cost function is crafted to sum up all relevant costs, directly linking our decision variables to the financial impact of different distribution strategies.

7. **Adding Constraints:** Constraints are methodically added to the problem, ensuring solutions remain feasible by respecting station capacities, demand requirements, and logistical considerations.

8. **Solving the Optimization:** With the objective and constraints in place, the `solve` method is invoked to find the optimal distribution of bicycles that minimizes costs while adhering to all operational constraints.

9. **Result Extraction:** Finally, upon solving the optimization model, the solution's status is assessed to ensure an optimal result was achieved or to identify any potential issues. Subsequently, the model provides detailed insights by extracting the values of decision variables, such as the optimal number of bikes per station, stock adjustments, and redistribution needs, enabling strategic management of the bike-sharing system. Finally, the minimized total cost is determined, reflecting the economic efficiency of the distribution strategy and guiding cost-effective operational decisions.

This Python implementation bridges the gap between our theoretical optimization model and practical decision-making, offering a systematic approach to managing bicycle distribution in a sharing network.

<span>

In [2]:
# Importing the PuLP library to define and solve linear optimization problems
import pulp

# Defining the bike sharing optimization model function
def bike_sharing_optimization_model(daily_demand, bike_stations, purchase_cost, 
                          min_stock_out_cost, max_stock_out_cost, 
                          min_time_waste_cost, max_time_waste_cost, 
                          min_redistribution_cost, max_redistribution_cost, 
                          min_station_capacity, max_station_capacity):

    # Assigning daily demand to a local variable
    daily_demand = daily_demand
    # Setting the purchase cost for each bike at every station
    cost_per_bike = {station: purchase_cost for station in range(bike_stations)}
    # Defining stock out costs randomly within specified bounds for each station
    stock_out_cost = {station: random.uniform(min_stock_out_cost, max_stock_out_cost) for station in range(bike_stations)}
    # Setting time waste costs randomly within specified bounds for each station
    time_waste_cost = {station: random.uniform(min_time_waste_cost, max_time_waste_cost) for station in range(bike_stations)}
    # Defining redistribution costs randomly within specified bounds for each pair of stations
    redistribution_cost = {(station1, station2): random.uniform(min_redistribution_cost, max_redistribution_cost) for station1 in range(bike_stations) for station2 in range(bike_stations) if station1 != station2}
    # Setting station capacities randomly within specified bounds for each station
    station_capacity = {station: random.randint(min_station_capacity, max_station_capacity) for station in range(bike_stations)}

    # Defining the optimization problem with the goal to minimize the total cost
    bike_sharing_problem = pulp.LpProblem("BikeSharingProblem", pulp.LpMinimize)

    # Creating decision variables for the number of bikes at each station
    bikes_at_station = pulp.LpVariable.dicts("BikesAtStation", range(bike_stations), lowBound=0, cat=pulp.LpInteger)
    # Decision variables for stock shortage between stations
    stock_shortage = pulp.LpVariable.dicts("StockShortage", (range(bike_stations), range(bike_stations)), lowBound=0, cat=pulp.LpInteger)
    # Decision variables for overstock at each station
    overstock = pulp.LpVariable.dicts("Overstock", range(bike_stations), lowBound=0, cat=pulp.LpInteger)
    # Decision variables for demand balance needed between stations
    demand_balance_needed = pulp.LpVariable.dicts("DemandBalanceNeeded", (range(bike_stations), range(bike_stations)), cat=pulp.LpInteger)
    # Decision variables for stock surplus at each station
    stock_surplus = pulp.LpVariable.dicts("StockSurplus", range(bike_stations), lowBound=0, cat=pulp.LpInteger)
    # Decision variables for understock at each station
    understock = pulp.LpVariable.dicts("Understock", range(bike_stations), lowBound=0, cat=pulp.LpInteger)
    # Decision variables for demand balance received between stations
    demand_balance_received = pulp.LpVariable.dicts("DemandBalanceReceived", (range(bike_stations), range(bike_stations)), lowBound=0, cat=pulp.LpInteger)
    # Decision variables for redistribution incoming to each station
    redistribution_in = pulp.LpVariable.dicts("RedistributionIn", range(bike_stations), lowBound=0, cat=pulp.LpInteger)
    # Decision variables for redistribution outgoing from each station
    redistribution_out = pulp.LpVariable.dicts("RedistributionOut", range(bike_stations), lowBound=0, cat=pulp.LpInteger)
    # Decision variables for redistribution volume between stations
    redistribution_volume = pulp.LpVariable.dicts("RedistributionVolume", (range(bike_stations), range(bike_stations)), lowBound=0, cat=pulp.LpInteger)

    # Constructing the objective function with costs associated with bikes, stock shortages, overstocks, and redistributions
    bike_sharing_problem += pulp.lpSum([cost_per_bike[station] * bikes_at_station[station] for station in range(bike_stations)]) + \
                            pulp.lpSum([stock_out_cost[station] * stock_shortage[station][other_station] for station in range(bike_stations) for other_station in range(bike_stations)]) + \
                            pulp.lpSum([time_waste_cost[station] * overstock[station] for station in range(bike_stations)]) + \
                            pulp.lpSum([redistribution_cost[(station1, station2)] * redistribution_volume[station1][station2] for station1 in range(bike_stations) for station2 in range(bike_stations) if station1 != station2])

    # Adding constraints to the problem for bike station capacities, demand balances, stock levels, and redistributions
    for station in range(bike_stations):
        # Ensuring bikes at each station do not exceed station capacity
        bike_sharing_problem += bikes_at_station[station] <= station_capacity[station], f"MaxBikes_{station}"

        # Balancing demand and supply between stations
        for other_station in range(bike_stations):
            if station != other_station:
                # Calculating demand balance needed based on daily demand and stock shortages
                bike_sharing_problem += demand_balance_needed[station][other_station] == daily_demand[station,other_station] - stock_shortage[station][other_station], f"DemandBalanceNeeded_{station}_{other_station}"
                # Computing stock surplus as the difference between bikes at station and the sum of daily demands adjusted for stock shortages
                bike_sharing_problem += stock_surplus[station] - pulp.lpSum([stock_shortage[station][other_station]]) == bikes_at_station[station] - \
                    pulp.lpSum(daily_demand[station,other_station]), f"StockSurplus_{station}_{other_station}"

        # Balancing overstock and understock at each station considering station capacity and demand balances
        bike_sharing_problem += overstock[station] - understock[station] == station_capacity[station] - bikes_at_station[station] + \
            pulp.lpSum([demand_balance_needed[station][other_station] for other_station in range(bike_stations)]) - \
                pulp.lpSum([demand_balance_needed[other_station][station] for other_station in range(bike_stations)]), f"Overstock_{station}"
        # Ensuring demand balance received matches understock at each station
        bike_sharing_problem += pulp.lpSum([demand_balance_received[station][other_station] for other_station in range(bike_stations)]) == understock[station], f"DemandBalanceReceived_{station}"
        # Limiting demand balance received to not exceed overstock at each station
        bike_sharing_problem += pulp.lpSum([demand_balance_received[other_station][station] for other_station in range(bike_stations)]) <= overstock[station], f"DemandBalanceReceivedLimit_{station}"
        # Ensuring net redistribution in matches the difference between station capacity and overstock adjusted for demand balance received and bikes at station
        bike_sharing_problem += redistribution_in[station] - redistribution_out[station] == station_capacity[station] - overstock[station] + \
            pulp.lpSum([demand_balance_received[station][other_station] for other_station in range(bike_stations)]) - bikes_at_station[station], f"RedistributionIn_{station}"
        # Matching total redistribution volume in with redistribution in for each station
        bike_sharing_problem += pulp.lpSum([redistribution_volume[station][other_station] for other_station in range(bike_stations)]) == redistribution_in[station], f"RedistributionVolume_{station}"
        # Ensuring total redistribution volume out matches redistribution out for each station
        bike_sharing_problem += pulp.lpSum([redistribution_volume[other_station][station] for other_station in range(bike_stations)]) == redistribution_out[station], f"RedistributionVolumeOut_{station}"

    # Solving the optimization problem and retrieving the status
    optimization_status = bike_sharing_problem.solve()

    # Extracting and organizing the results from the solved problem, including status, total cost, and decision variable values
    results = {"Status": pulp.LpStatus[optimization_status]}
    results["TotalCost"] = pulp.value(bike_sharing_problem.objective)
    for station in range(bike_stations):
        results[f"BikesAtStation_{station}"] = bikes_at_station[station].value()
        results[f"StockSurplus_{station}"] = stock_surplus[station].value()
        results[f"Overstock_{station}"] = overstock[station].value()
        results[f"RedistributionIn_{station}"] = redistribution_in[station].value()
        results[f"RedistributionOut_{station}"] = redistribution_out[station].value()
        for other_station in range(bike_stations):
            results[f"StockShortage_{station}_{other_station}"] = stock_shortage[station][other_station].value()
            results[f"DemandBalanceNeeded_{station}_{other_station}"] = demand_balance_needed[station][other_station].value()
            results[f"DemandBalanceReceived_{station}_{other_station}"] = demand_balance_received[station][other_station].value()
            results[f"RedistributionVolume_{station}_{other_station}"] = redistribution_volume[station][other_station].value()

    # Returning the organized results
    return results

<span style="font-family:Times New Roman; font-size:15pt;">
<h5><b>4.3&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Further Discussion<b></h4>
<span>

<span style="font-family: Times New Roman; font-size: 13pt;">

Our objective function balances operational costs with those related to customer satisfaction. This dual focus ensures we pursue efficiency and cost-effectiveness while minimizing user dissatisfaction. Integrating operational costs (procurement, redistribution) with behavioral costs (stockouts, time wastage) creates a holistic view of the bike-sharing ecosystem, essential for maintaining service quality and long-term success.

To enhance realism, we introduced station-specific cost variability, excluding procurement. Factors like location, demographics, and infrastructure cause operational costs to fluctuate within predefined bounds. This mirrors real-world complexity, increasing the model's applicability across diverse urban environments and strengthening strategic decision-making.

The model is grounded in foundational research. Insights from Maggioni et al. (2019) on stochastic optimization with transshipment and Raviv et al. (2013) on static repositioning provide the theoretical and empirical basis for our approach, incorporating advanced techniques to navigate system management complexities.
<span>

<span style="font-family:Times New Roman; font-size:15pt;">
<h3><b>5&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Section Three; Solving for Optimal Bicycle Distribution with Forecasted Demand<b></h4>
<span>
<span style="font-family: Times New Roman; font-size: 13pt;">

With our optimization problem fully modeled and implemented, we now determine the optimal bicycle allocation across the network. This final step applies our theoretical framework to practical, data-driven decision-making.

The process begins with demand inputs for each station, generated by our distribution function. This function takes total daily demand and systematically allocates it across stations. These total demand figures come from the Random Forest forecasts from the previous phase, aggregated for the upcoming seven days.

Using these figures, a for loop iteratively solves for the most efficient daily bicycle allocation with our optimization model. This dynamic approach adapts to fluctuating demand, ensuring optimal supply and availability across the network.

Finally, we statistically analyze the model's outputs to distill actionable insights for strategic bicycle placement. This methodical approach enhances operational efficiency and user satisfaction by ensuring bicycles are optimally distributed to meet forecasted demand.

<span>

<span style="font-family:Times New Roman; font-size:15pt;">
<h5><b>5.1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Total Daily Demands<b></h4>
<span>

<span style="font-family: Times New Roman; font-size: 13pt;">

In this initial phase of solving for the optimal bicycle allocation, our primary objective is to prepare the demand inputs necessary for our optimization function. This involves transforming the forecasted data from the previous phase into a structured format that represents the total daily demand for bicycles. By accurately aggregating the forecasted demand for each day, we can ensure that our optimization model is informed by realistic and data-driven demand patterns, setting the stage for effective and efficient bicycle distribution decisions.

The code snippet provided performs several key operations to achieve this goal:

1. **Importing Libraries and Data:**
   - The code begins by importing the `pandas` library, which is essential for data manipulation and analysis in Python. It then loads the forecasted data from a CSV file into a DataFrame named `Forecasted_Data`, providing a structured representation of the forecasts generated in the previous phase.

2. **Data Preprocessing:**
   - The `rename` method is used to rename the unnamed column to 'Date', ensuring clarity and consistency in the DataFrame's structure. This is followed by converting the 'Date' column to a datetime object using `pd.to_datetime`, which facilitates date-based operations in the subsequent steps.

3. **Aggregating Daily Demand:**
   - The DataFrame is grouped by the date, and the forecasted values under the 'Random Forest' column are summed up to represent the total forecasted demand for bicycles on each day. The `round` method is applied to ensure that the demand values are rounded to the nearest integer, reflecting the discrete nature of bicycles.

4. **Creating the Demand Dictionary:**
   - The aggregated daily demand is iterated over using a for loop, and for each date, the total demand is extracted and stored in a dictionary named `daily_demands`. This dictionary maps each date to its corresponding total daily demand, providing a concise and accessible format for the demand data.

By meticulously preparing the demand data in this manner, we lay a solid foundation for the subsequent optimization steps, ensuring that our model operates on accurate and comprehensive demand forecasts. This preparation is crucial for enabling the optimization function to determine the most effective allocation of bicycles, thereby enhancing the overall efficiency and user satisfaction of the bike-sharing system.

<span>

In [3]:
import pandas as pd  # Importing the pandas library for data manipulation and analysis.

# Loading forecasted demand data from a CSV file into a pandas DataFrame.
Forecasted_Data = pd.read_csv(r"G:\University\Terme 9\Assignment\TP\Project\Phase 2\Forecasted_Data.csv")

# Renaming the first unnamed column to 'Date' for better readability and consistency.
Forecasted_Data.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

# Converting the entries in the 'Date' column from strings to datetime objects for easier date manipulation.
Forecasted_Data['Date'] = pd.to_datetime(Forecasted_Data['Date'])

# Aggregating forecasted demands by date, summing up the demands predicted by the Random Forest model, and rounding the results to the nearest whole number.
daily_demand = Forecasted_Data.groupby(Forecasted_Data['Date'].dt.date)['Random Forest'].sum().round().reset_index()

# Initializing an empty dictionary to hold the total daily demand for each date.
daily_demands = {}

# Iterating through each row of the aggregated demand DataFrame to populate the 'daily_demands' dictionary with date as key and total demand as value.
for index, row in daily_demand.iterrows():
    date = str(row['Date'])  # Converting the date to a string format for use as a dictionary key.
    total_demand = row['Random Forest']  # Extracting the total demand for the day from the 'Random Forest' column.
    daily_demands[date] = total_demand  # Assigning the total demand to the corresponding date in the dictionary.

<span style="font-family:Times New Roman; font-size:15pt;">
<h5><b>5.2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Solving the Optimization Model<b></h4>
<span>

<span style="font-family: Times New Roman; font-size: 13pt;">

This code segment applies forecasted daily demands to our optimization model, translating theoretical predictions into actionable bike distribution strategies. This phase ensures the system meets user demand while minimizing costs, enhancing service quality and satisfaction.

**Parameter Setup:**
The code establishes constant parameters defining the operational environment: number of stations, bike purchase costs, and ranges for stockout, time wastage, and redistribution costs. Minimum and maximum values for each cost parameter are drawn from real-world analyses, capturing operational variability. This assigns each station a unique cost within realistic bounds, adding fidelity and ensuring robust, applicable outcomes.

**Purchase Cost Sensitivity:**
The model is highly sensitive to bike purchase cost. Setting it above 5 dollars per bike often results in zero allocations across all stations. This occurs because purchase cost is aggregated while other costs are individual-based. To address this, we amortize the bike cost over the number of users (e.g., a 200 dollars bike used by 100 people costs 2 dollars per use). This aligns costs and enables realistic, actionable allocations.

**Iterative Optimization:**
Iterating over forecasted daily demands, the code dynamically distributes demand across stations using the distribution function with a normal distribution. This simulates daily demand variation, providing detailed inputs for optimization. The `bike_sharing_optimization_model` function then solves for each day, determining optimal bike allocations that balance meeting demand with minimizing stockout, excess inventory, and redistribution costs.

**Results Compilation:**
Optimization outputs—optimal bike allocations per station, daily total cost, and run status—are compiled into a DataFrame. This structured format provides a snapshot of each day's strategy and a foundation for in-depth analysis. Examining these outcomes yields strategic insights for fleet management, ensuring responsiveness and efficiency.

This methodical approach, grounded in real-world costs and accurate forecasts, exemplifies data-driven optimization for complex service systems. It enhances operational efficiency and contributes to a sustainable, user-centric transportation solution.
<span>

In [4]:
# Defining constant parameters for the optimization, applicable to all simulation runs
bike_stations = 5  # Setting the number of bike stations
purchase_cost = 2  # Setting the cost to purchase a bike
min_stock_out_cost = 5  # Setting the minimum cost incurred from a stock out
max_stock_out_cost = 15  # Setting the maximum cost incurred from a stock out
min_time_waste_cost = 2  # Setting the minimum cost associated with time wasted due to excess inventory
max_time_waste_cost = 8  # Setting the maximum cost associated with time wasted due to excess inventory
min_redistribution_cost = 1  # Setting the minimum cost of redistributing bikes between stations
max_redistribution_cost = 5  # Setting the maximum cost of redistributing bikes between stations
min_station_capacity = 20  # Setting the minimum capacity of a bike station
max_station_capacity = 40  # Setting the maximum capacity of a bike station

# Initializing a DataFrame to store the results of each run with relevant columns
results_df = pd.DataFrame(columns=['Date', 'Total Demand'] + [f'Bikes in Station {i+1}' for i in range(bike_stations)] + ['Total Cost'] + ['Status'])

# Iterating over each date and its corresponding total demand in the daily demands dictionary
for date, total_demand in daily_demands.items():
    daily_demand = daily_demand_distributor(total_demand, bike_stations, 'normal')  # Distributing the total demand among stations based on a normal distribution
    results = bike_sharing_optimization_model(daily_demand, bike_stations, purchase_cost, 
                          min_stock_out_cost, max_stock_out_cost, 
                          min_time_waste_cost, max_time_waste_cost, 
                          min_redistribution_cost, max_redistribution_cost, 
                          min_station_capacity, max_station_capacity)  # Optimizing bike allocation for the given day's demand

    # Constructing a row for the results DataFrame from the optimization output
    result_row = [date, total_demand] + list({key: value for key, value in results.items() if key.startswith('BikesAtStation')}.values()) + [round(results['TotalCost'],3)] + [results['Status']]
    results_df.loc[len(results_df)] = result_row  # Adding the constructed row to the results DataFrame

# Displaying the completed results DataFrame in a format that's easy to read
results_df

,Date,Total Demand,Bikes in Station 1,Bikes in Station 2,Bikes in Station 3,Bikes in Station 4,Bikes in Station 5,Total Cost,Status
0,2018-12-01,3895.0,39.0,33.0,36.0,29.0,30.0,40780.566,Optimal
1,2018-12-02,4334.0,21.0,27.0,24.0,35.0,25.0,39507.918,Optimal
2,2018-12-03,6324.0,32.0,40.0,20.0,21.0,27.0,60147.323,Optimal
3,2018-12-04,6338.0,22.0,30.0,27.0,34.0,37.0,58006.789,Optimal
4,2018-12-05,5971.0,37.0,23.0,34.0,28.0,37.0,56693.442,Optimal
5,2018-12-06,4404.0,31.0,34.0,23.0,27.0,27.0,38661.659,Optimal
6,2018-12-07,4742.0,32.0,33.0,27.0,36.0,31.0,39653.939,Optimal


<span style="font-family:Times New Roman; font-size:15pt;">
<h4><b><b></h4>
<span>

<span style="font-family: Times New Roman; font-size: 13pt;">

While our optimization model generates daily allocation strategies, daily adjustments are impractical for real-world management. We need a single, overarching seven-day strategy. Integrating time directly into the model would add unnecessary complexity, obscuring clear insights.

A more elegant solution involves calculating a weighted average of bikes per station over the seven-day period, using daily total demand as weights. This method acknowledges demand variability while producing a single, manageable allocation figure, streamlining decision-making without oversimplifying underlying patterns.

To ensure robustness, we analyze outcomes across different demand distribution patterns, identifying a general allocation approach that remains effective under diverse conditions. This comprehensive analysis enables an informed decision on the optimal number of bikes to purchase and allocate per station. The resulting strategy balances fluctuating daily demands with operational efficiency, optimizing resource use and enhancing system responsiveness for a more reliable, user-friendly service.

<span>

<span style="font-family: Times New Roman; font-size: 13pt;">

The optimization process successfully solved all forecasted days, each reaching an optimal status. This demonstrates our linear model's efficacy in navigating bike-sharing complexities and finding efficient allocation strategies. Its consistent optimal solutions prove robustness, offering valuable operational insights.

The model's flexibility allows easy adjustment of parameters like station numbers and costs to fit different scenarios. This adaptability makes it a valuable tool for exploring strategies and ensuring responsiveness to changing demands.

The demand distribution method significantly impacts outcomes. Currently normal, this parameter can be changed to explore alternative methods and observe their effect on allocations, enhancing our understanding of how demand variability influences resource allocation.

<span>

<span style="font-family:Times New Roman; font-size:15pt;">
<h4><b>5.3&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Deriving a Managerial Decision<b></h4>
<span>

<span style="font-family: Times New Roman; font-size: 13pt;">

To translate daily optimization results into a practical management strategy, we need a consolidated approach rather than daily adjustments. A simple solution is calculating a weighted average of bikes per station over the seven-day period, using daily total demand as weights. This captures demand variability while providing a single, actionable allocation figure.

To ensure robustness, we analyze outcomes across different demand distribution patterns, identifying a strategy effective under diverse conditions. This analysis informs decisions on optimal bike purchases and allocations per station, balancing fluctuating demand with operational efficiency and enhancing service reliability.

<span>

<span style="font-family: Times New Roman; font-size: 13pt;">
<b>

<span style="font-family:Times New Roman; font-size:13pt;">
<h5><b>5.3.1&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Code Implementation<b></h4>
<span>

The code below is exactly designed to do so. Like the last part, it optimizes the allocation of bicycles across a network of bike-sharing stations; But does it by applying different demand distribution methods and then summarizing the results in a way that informs managerial decisions, meaning one singule number of bikes needed for each station in general. Here's a detailed explanation of the code:

1. **Setting Constants:**
   The code, again, begins by defining a set of constant parameters that are critical to the optimization model. These include the number of bike stations in the network (the number of stations is chosen to be 15 to show even for a problem with higher dimension, this code works like butter), the cost associated with purchasing a bike, the range of costs related to stockouts and time wasted due to excess inventory, the costs for redistributing bikes between stations, and the minimum and maximum capacities for each station. These constants provide the framework within which the optimization model operates.  

2. **Distribution Methods:**
   A list named `distribution_methods` is defined, containing various statistical methods ('normal', 'lognormal', 'uniform', 'exponential', 'poisson') that will be used to distribute the forecasted demand for bicycles among the stations. This allows the model to simulate different real-world scenarios where demand distribution might vary according to different statistical patterns.

3. **Optimization Loop:**
   The code iterates over each distribution method. For each method, it initializes a DataFrame `results_df` to store the optimization results, which include the date, total demand, the number of bikes allocated to each station, the total cost of the allocation, and the status of the optimization process (e.g., 'Optimal').

4. **Daily Demand Distribution and Optimization:**
   Within each iteration of the distribution methods, the code further iterates over a dictionary `daily_demands`, which contains the total forecasted demand for bikes on each day. For each day, the `daily_demand_distributor` function is called to distribute the total demand among the stations according to the current distribution method. The `bike_sharing_optimization_model` function then takes these distributed demands and the constant parameters to solve the optimization problem, determining the best allocation of bikes for that day to minimize total costs.

5. **Constructing Results Rows:**
   After solving the optimization problem for a given day and distribution method, the code constructs a row of results, including the date, total demand, the optimized number of bikes at each station, the total cost, and the optimization status. This row is appended to the `results_df` DataFrame.

6. **Storing Results:**
   Once all days have been processed for a particular distribution method, the completed `results_df` DataFrame is stored in the `results_dfs` dictionary, keyed by the distribution method.

7. **Calculating Weighted Averages:**
   After processing all distribution methods, the code initializes another DataFrame `weighted_bikes_df` with distribution methods as rows and stations as columns. It then calculates the weighted average number of bikes for each station across all days, using the total demand on each day as weights. This calculation is performed for each distribution method, providing insight into how many bikes should be allocated to each station under different demand scenarios.

8. **Displaying Results:**
   Finally, the `weighted_bikes_df` DataFrame, which summarizes the weighted average number of bikes for each station under each distribution method, is printed. This summary offers valuable insights for making managerial decisions regarding the general allocation of bikes to stations, considering different demand distribution patterns.

This comprehensive approach allows for a nuanced understanding of how different demand distributions affect bike allocation strategies, facilitating informed, data-driven decision-making in managing the bike-sharing system.
<b>
<span>

In [5]:
# Defining constant parameters for the optimization, applicable to all simulation runs
bike_stations = 15  # Setting the number of bike stations
purchase_cost = 2  # Setting the cost to purchase a bike
min_stock_out_cost = 5  # Setting the minimum cost incurred from a stock out
max_stock_out_cost = 15  # Setting the maximum cost incurred from a stock out
min_time_waste_cost = 2  # Setting the minimum cost associated with time wasted due to excess inventory
max_time_waste_cost = 8  # Setting the maximum cost associated with time wasted due to excess inventory
min_redistribution_cost = 1  # Setting the minimum cost of redistributing bikes between stations
max_redistribution_cost = 5  # Setting the maximum cost of redistributing bikes between stations
min_station_capacity = 20  # Setting the minimum capacity of a bike station
max_station_capacity = 40  # Setting the maximum capacity of a bike station

# Define your distribution methods
distribution_methods = ['normal', 'lognormal', 'uniform', 'exponential', 'poisson']

# Initialize an empty dictionary to store results DataFrames for each distribution method
results_dfs = {}

# Loop over each distribution method to generate and store optimization results
for method in distribution_methods:
    # Initialize a DataFrame to store the results for the current distribution method
    results_df = pd.DataFrame(columns=['Date', 'Total Demand'] + [f'Bikes in Station {i+1}' for i in range(bike_stations)] + ['Total Cost', 'Status'])
    
    # Iterate over each day's demand in your daily_demands dictionary
    for date, total_demand in daily_demands.items():

        daily_demand = daily_demand_distributor(total_demand, bike_stations, method)  # Distributing the total demand among stations based on a normal distribution
        results = bike_sharing_optimization_model(daily_demand, bike_stations, purchase_cost, 
                          min_stock_out_cost, max_stock_out_cost, 
                          min_time_waste_cost, max_time_waste_cost, 
                          min_redistribution_cost, max_redistribution_cost, 
                          min_station_capacity, max_station_capacity)  # Optimizing bike allocation for the given day's demand
        
        # Construct a row with the results and add it to the DataFrame
        result_row = [date, total_demand] + list({key: value for key, value in results.items() if key.startswith('BikesAtStation')}.values()) + [round(results['TotalCost'],3)] + [results['Status']]
        results_df.loc[len(results_df)] = result_row  # Adding the constructed row to the results DataFrame
    
    # Store the completed DataFrame in the results_dfs dictionary
    results_dfs[method] = results_df

# Assuming results_dfs is a dictionary containing results_df for each distribution method
distribution_methods = ['normal', 'lognormal', 'uniform', 'exponential', 'poisson']  # Example distribution methods
weighted_bikes_df = pd.DataFrame(index=distribution_methods, columns=[f'Station {i+1}' for i in range(bike_stations)])

for method in distribution_methods:
    results_df = results_dfs[method]  # Retrieve the results DataFrame for the current distribution method

    # Calculate the weighted average of bikes for each station
    for i in range(bike_stations):
        station_column = f'Bikes in Station {i+1}'
        weighted_bikes = (results_df[station_column] * results_df['Total Demand']).sum() / results_df['Total Demand'].sum()
        weighted_bikes_df.loc[method, f'Station {i+1}'] = round(weighted_bikes)

# Displaying the completed data in a format that's easy to read
weighted_bikes_df

,Station 1,Station 2,Station 3,Station 4,Station 5,Station 6,Station 7,Station 8,Station 9,Station 10,Station 11,Station 12,Station 13,Station 14,Station 15
normal,31,31,30,27,34,30,30,31,30,30,31,31,32,29,31
lognormal,30,34,28,32,28,28,32,28,29,33,32,30,27,33,33
uniform,34,27,25,35,29,30,29,30,31,33,29,32,27,31,29
exponential,29,25,29,30,30,29,31,34,32,30,31,29,28,28,29
poisson,33,30,28,29,30,29,30,32,30,32,29,31,30,30,32


<span style="font-family: Times New Roman; font-size: 13pt;">
<b>

<span style="font-family:Times New Roman; font-size:13pt;">
<h5><b>5.3.2&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Results & Managerial Insights<b></h4>
<span>

The final result presents the weighted average number of bikes allocated to each of the 15 stations under different demand distribution methods: normal, lognormal, uniform, exponential, and Poisson. Analyzing this data can provide valuable insights into how the variability in demand affects the strategic allocation of bikes across the network. Here's a detailed analysis:

1. **Demand Sensitivity:** The variation in bike allocation across different distribution methods indicates the model's sensitivity to demand patterns. For instance, the 'normal' and 'lognormal' distributions, which are more likely to represent real-world demand scenarios with a central tendency, show a relatively balanced distribution of bikes across stations. In contrast, 'uniform' distribution, which assumes equal probability for all demand levels, results in a more even spread of bikes.

2. **Station-Specific Allocations:** Certain stations exhibit higher allocations across most distribution methods, suggesting these are key hubs with consistently higher demand. In contrast, some stations (e.g., Station 7 and Station 13) generally receive fewer bikes, which might be due to their location, lesser demand, or proximity to other high-demand stations.

3. **Impact of Extreme Values:** The 'exponential' and 'poisson' distributions, which can include more extreme values due to their long tails, show significant variability in bike allocations. This is evident from the higher allocations in some stations under the 'poisson' method, reflecting the model's attempt to accommodate sporadic high-demand events.

4. **Operational Implications:** The variability in allocations underscores the need for a flexible operational strategy that can adapt to daily demand fluctuations. Stations with consistently higher allocations should be prioritized for regular monitoring and rebalancing to ensure availability. Conversely, stations with lower allocations might require less frequent interventions but should not be neglected, as their demand could increase due to special events or seasonal changes.

5. **Strategic Planning:** The data provides a foundation for long-term strategic planning, such as deciding where to place new stations or which stations to expand. Stations consistently receiving high allocations across various distributions might be candidates for capacity increase or infrastructure improvements.

6. **Resource Optimization:** Understanding the demand distribution's impact on bike allocation helps optimize resources, ensuring that bikes and maintenance efforts are directed where they are most needed, enhancing system efficiency and user satisfaction.

7. **Future Investments:** For stations showing high variability in bike allocations depending on the demand distribution, further investigation into local factors influencing demand might be necessary before making significant investments.

This analysis highlights the importance of considering various demand scenarios in planning and operating a bike-sharing system. It also underscores the need for dynamic management practices that can adjust to changing demand patterns to maintain a high level of service.


<b>
<span>

<span style="font-family:Times New Roman; font-size:15pt;">
<h3><b>6&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Conclusion<b></h4>
<span>
<span style="font-family: Times New Roman; font-size: 13pt;">

In conclusion, this project has adeptly navigated the complexities of optimizing bicycle allocations for a bike-sharing network, underpinned by robust demand forecasts generated through a Random Forest algorithm. Central to our endeavor was the development of a comprehensive mathematical optimization model, which meticulously balanced various operational costs against the dynamic landscape of forecasted demand. This model not only aimed at minimizing operational expenses but also ensured that bike distribution was finely tuned to meet anticipated demand fluctuations, highlighting the critical interplay between predictive analytics and operational strategy.

The exploration of diverse demand distribution methods further enriched our model's adaptability, reflecting the real-world variability inherent in bike-sharing systems. This nuanced approach facilitated a deeper understanding of demand dynamics, enabling the identification of key operational insights such as high-demand hubs and stations with lower allocation needs. Such insights are invaluable for targeted resource allocation, enhancing system efficiency and user satisfaction.

Moreover, the project's findings lay a solid foundation for future strategic planning within bike-sharing operations. The adaptability of our optimization framework to various demand scenarios and different parameters offers a powerful tool for informed decision-making, guiding the expansion and efficient management of bike-sharing networks. As urban mobility continues to evolve, the methodologies and insights gleaned from this project will remain pertinent, steering the sustainable and user-centric development of bike-sharing systems.

<span>

